***
*Project:* Helmholtz Machine on Niche Construction

*Author:* Jingwei Liu, Computer Music Ph.D., UC San Diego

*Supervisor:* Shlomo Dubnov, Professor in Music and CSE department, UC San Diego
***

# <span style="background-color:darkorange; color:white; padding:2px 6px">Document 4</span> 

# Helmholtz Machine Implementation

*Updated:* May 24, 2023


In [1]:
import numpy as np

### 1. Ground Truth

In [2]:
n = 10
preferred_set = np.zeros([1,10])
preferred_set[0,:2] = 1
preferred_set[0,-2:] = 1

for i in range(2, n-2):
    for j in range(np.shape(preferred_set)[0]):
        
        prefix = preferred_set[j,i-2:i]
        if np.array_equal(prefix, [0,0]) or np.array_equal(prefix, [0,1]):
            preferred_set[j,i] = 1
        else:
            preferred_set = np.append(preferred_set, preferred_set[j:j+1,:], axis=0)
            preferred_set[j,i] = 1
preferred_set = (preferred_set - 0.5)*2
preferred_set

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.],
       [ 1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.],
       [ 1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.],
       [ 1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.],
       [ 1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.],
       [ 1.,  1., -1.,  1.,  1., -1., -1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.],
       [ 1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.

In [3]:
np.shape(preferred_set)

(25, 10)

There are 25 elements in the preferred set.

In [4]:
well_formed_set = np.zeros([1,10])
well_formed_set[0,0] = 1

for i in range(1,n):
    for j in range(np.shape(well_formed_set)[0]):
        if i == 2 and np.array_equal(well_formed_set[j,i-2:i], [1,0]):
            well_formed_set[j,i] = 1
        elif i > 3 and np.array_equal(well_formed_set[j,i-3:i], [0,0,0]):
            well_formed_set[j,i] = 1
        elif i > 3 and np.array_equal(well_formed_set[j,i-4:i], [0,0,1,0]):
            well_formed_set[j,i] = 1
        else:
            well_formed_set = np.append(well_formed_set, well_formed_set[j:j+1,:], axis=0)
            well_formed_set[j,i] = 1
            
ind = np.array([], dtype=np.int8)
for i in range(well_formed_set.shape[0]):
    if np.array_equal(well_formed_set[i,-3:], [0,0,1]):
        ind = np.append(ind,i)

well_formed_set = np.delete(well_formed_set,ind,0)
well_formed_set = (well_formed_set - 0.5)*2
well_formed_set

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1., -1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1., -1., ...,  1.,  1.,  1.],
       ...,
       [ 1.,  1.,  1., ..., -1., -1., -1.],
       [ 1., -1.,  1., ..., -1., -1., -1.],
       [ 1.,  1., -1., ..., -1., -1., -1.]])

In [5]:
well_formed_set.shape

(256, 10)

There are 256 elements in the preferred set.

There are $2^{10} = 1024$ possible sequences in total, thus the Venn diagram for the ground truth is 

<img src="Venn_1.jpg" style="width:550px;height:500px;">
<caption><center> **Figure 2**: Venn Diagram for Ground Truth Data  </center></caption>

### 2. Basic Functions

The Helmholtz machine is designed as follows

<img src="niche.jpg">
<caption><center> **Figure 1**: Helmholtz Machine in Niche Construction  </center></caption>

where

- Input layer $d_i$ with 10 neurons $i = 1, \dots, 10, 2^{10} = 1024$ possibilities
- Hidden layer $x_l$ wiht 8 neurons $l = 1, \dots, 8, 2^{8} = 256$ possibilities
- Cause layer $y_j$ wiht 5 neurons $j = 1, \dots, 5, 2^{5} = 32$ possibilities
- Hyper layer $z_k$ wiht 3 neurons $k = 1, \dots, 3, 2^{3} = 8$ possibilities
- Generative bias is always $1$ (**Ursatz** in Schenkerian analysis)

#### 2.1 Parameter Initialization

In [6]:
n_z = 3
n_y = 5
n_x = 8
n_d = 10

In [275]:
Phi_12 = np.random.rand(n_x,n_d) *2-1
Phi_23 = np.random.rand(n_y,n_x) *2-1
Phi_34 = np.random.rand(n_z,n_y) *2-1
b_12 = np.random.rand(n_x,1) *2-1
b_23 = np.random.rand(n_y,1) *2-1
b_34 = np.random.rand(n_z,1) *2-1

Theta = np.random.rand(n_z,1) *2-1
Theta_43 = np.random.rand(n_y,n_z) *2-1
Theta_32 = np.random.rand(n_x,n_y) *2-1
Theta_21 = np.random.rand(n_d,n_x) *2-1
b_43 = np.random.rand(n_y,1) *2-1
b_32 = np.random.rand(n_x,1) *2-1
b_21 = np.random.rand(n_d,1) *2-1

In [276]:
print ("Phi_12: " + str(Phi_12))
print ("Phi_23: " + str(Phi_23))
print ("Phi_34: " + str(Phi_34))
print ("b_12: " + str(b_12))
print ("b_23: " + str(b_23))
print ("b_34: " + str(b_34))

Phi_12: [[ 0.70046353  0.66337122  0.81162809  0.91505913 -0.70999478 -0.22892445
  -0.43162401 -0.24685352  0.96928902 -0.59891461]
 [-0.30175166  0.90167623 -0.35749612  0.60922608  0.08334442 -0.5727393
  -0.47784762  0.03597584 -0.11936152 -0.56561284]
 [ 0.30065009  0.49038863  0.49222799  0.57652788 -0.18599646 -0.35060745
   0.41602025 -0.82638127 -0.00819545 -0.80736668]
 [-0.39938742 -0.46000916 -0.85419421  0.31628239 -0.71688841 -0.10476997
  -0.42615797 -0.52666339  0.4706374   0.18462482]
 [-0.57058909 -0.34781187  0.42768421  0.22045128 -0.3711668  -0.44448819
   0.86315534  0.94549712  0.39262227 -0.12812379]
 [ 0.72423593  0.63887754  0.08258646 -0.97875025  0.87371566 -0.66692262
  -0.47107942  0.36840904 -0.68557307 -0.52381884]
 [ 0.50127086 -0.49757956 -0.21973637 -0.59182289 -0.58686203  0.96174708
  -0.45732905 -0.46815264 -0.06773985  0.01557436]
 [ 0.35883974  0.6990014  -0.99567695 -0.34225972 -0.59472645 -0.11087274
  -0.05386368 -0.94540727  0.22035887 -0.848

In [277]:
print ("Theta: " + str(Theta))
print ("Theta_43: " + str(Theta_43))
print ("Theta_32: " + str(Theta_32))
print ("Theta_21: " + str(Theta_21))
print ("b_43: " + str(b_43))
print ("b_32: " + str(b_32))
print ("b_21: " + str(b_21))

Theta: [[ 0.54095459]
 [-0.02694901]
 [ 0.26798225]]
Theta_43: [[-0.28419503  0.62096252  0.31468793]
 [ 0.84400597  0.89836389 -0.0661224 ]
 [ 0.93881866 -0.97329526  0.5959983 ]
 [ 0.2987372   0.59434997  0.02154112]
 [-0.87552412 -0.13742513 -0.53410832]]
Theta_32: [[ 0.00257584  0.14395475 -0.43012178  0.16706566  0.68971692]
 [ 0.49041492 -0.24524771 -0.76753757 -0.53633693  0.56715938]
 [ 0.83413542  0.36312199 -0.96959998 -0.48311253  0.86106507]
 [ 0.19504268 -0.83019175  0.83037298 -0.44144708 -0.11396123]
 [-0.6047336   0.42420133  0.46086408  0.08561609  0.70286733]
 [ 0.39698949 -0.8442142  -0.48116    -0.57128674 -0.02298249]
 [-0.75458329  0.36702767  0.80536683  0.28457257  0.16978723]
 [ 0.23140572  0.96043924 -0.33133748  0.22099877  0.24382105]]
Theta_21: [[ 0.84309062  0.6799303   0.84063225 -0.09507744 -0.65524239 -0.94284151
   0.66691121 -0.43965246]
 [ 0.92247762 -0.55005819 -0.20365569 -0.0145644  -0.56005376  0.17333551
   0.7586334  -0.62438069]
 [ 0.83076873 

In [278]:
b_12.shape

(8, 1)

In [201]:
batch_size = 10

Total number of parameters: 135 + 16 + 138 + 23 = 312

#### 2.2 Function Definition

In [28]:
def sigmoid(x):
    y = 1/(1+np.exp(-x))
    return y

- As analyzed in *Document 2*, we use binary representatioin {1,-1} instead of {0,1} to replace the local delta rule where gradients vanish when the neuron takes value 0
- We use **rejection sampling** for each layer to choose from -1 or 1 (for computation efficiency)

In [225]:
def wake_forward(data,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,n_x,n_y,n_z):
    q_2 = sigmoid(np.matmul(Phi_12,data) + b_12)
    x = ((q_2 > np.random.rand(n_x,1)).astype(int) - 0.5)*2    # rejection sampling
    
    q_3 = sigmoid(np.matmul(Phi_23,x) + b_23)
    y = ((q_3 > np.random.rand(n_y,1)).astype(int) - 0.5)*2
    
    q_4 = sigmoid(np.matmul(Phi_34,y) + b_34)
    z = ((q_4 > np.random.rand(n_z,1)).astype(int) - 0.5)*2
    
    Q_2 = (np.cumprod(q_2[np.where(x == 1)])[-1] if q_2[np.where(x == 1)].size != 0 else 1) * (np.cumprod(1-q_2[np.where(x == -1)])[-1] if q_2[np.where(x == -1)].size != 0 else 1)*(1.5**n_x)
    Q_3 = (np.cumprod(q_3[np.where(y == 1)])[-1] if q_3[np.where(y == 1)].size != 0 else 1) * (np.cumprod(1-q_3[np.where(y == -1)])[-1] if q_3[np.where(y == -1)].size != 0 else 1)*(1.5**n_y)
    Q_4 = (np.cumprod(q_4[np.where(z == 1)])[-1] if q_4[np.where(z == 1)].size != 0 else 1) * (np.cumprod(1-q_4[np.where(z == -1)])[-1] if q_4[np.where(z == -1)].size != 0 else 1)*(1.5**n_z)
    Q = Q_2 * Q_3 * Q_4
    
    return q_2,q_3,q_4,x,y,z,Q

In [297]:
data = np.transpose(well_formed_set[0:1])
data

array([[ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.]])

In [222]:
q_2 = sigmoid(np.matmul(Phi_12,data) + b_12)
q_2

array([[0.03403704],
       [0.65332594],
       [0.61558687],
       [0.44143854],
       [0.69878497],
       [0.37753688],
       [0.0861012 ],
       [0.69294601]])

In [224]:
x = ((q_2 > np.random.rand(n_x,1)).astype(int) - 0.5)*2
x

array([[-1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.]])

In [285]:
# Find a cause

q_2,q_3,q_4,x_w,y_w,z_w,Q = wake_forward(data,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,n_x,n_y,n_z)
print ("q_2: " + str(q_2))
print ("q_3: " + str(q_3))
print ("q_4: " + str(q_4))
print ("x_w: " + str(x_w))
print ("y_w: " + str(y_w))
print ("z_w: " + str(z_w))
print ("Q: " + str(Q))

q_2: [[0.13352257]
 [0.746527  ]
 [0.81201678]
 [0.13464848]
 [0.22199253]
 [0.96318306]
 [0.51277176]
 [0.96457059]]
q_3: [[0.5756663 ]
 [0.97586899]
 [0.11609115]
 [0.90562918]
 [0.166656  ]]
q_4: [[0.55580529]
 [0.65498855]
 [0.04340911]]
x_w: [[-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]]
y_w: [[ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]]
z_w: [[-1.]
 [ 1.]
 [-1.]]
Q: 10.966231903455345


In [286]:
def wake_forward_batch(data,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,n_x,n_y,n_z,batch_size):
    q_2 = sigmoid(np.matmul(Phi_12,data) + b_12)
    x = ((q_2 > np.random.rand(n_x,batch_size)).astype(int) - 0.5)*2    # rejection sampling
    
    q_3 = sigmoid(np.matmul(Phi_23,x) + b_23)
    y = ((q_3 > np.random.rand(n_y,batch_size)).astype(int) - 0.5)*2
    
    q_4 = sigmoid(np.matmul(Phi_34,y) + b_34)
    z = ((q_4 > np.random.rand(n_z,batch_size)).astype(int) - 0.5)*2
    
    Q_2 = np.cumprod(np.where(x == 1,q_2,1),axis=0)[-1] * np.cumprod(np.where(x == -1,1-q_2,1),axis=0)[-1]*(1.5**n_x)
    Q_3 = np.cumprod(np.where(y == 1,q_3,1),axis=0)[-1] * np.cumprod(np.where(y == -1,1-q_3,1),axis=0)[-1]*(1.5**n_y)
    Q_4 = np.cumprod(np.where(z == 1,q_4,1),axis=0)[-1] * np.cumprod(np.where(z == -1,1-q_4,1),axis=0)[-1]*(1.5**n_z)
    Q = Q_2 * Q_3 * Q_4
    
    return q_2,q_3,q_4,x,y,z,Q

In [298]:
data_batch = np.transpose(well_formed_set[0:10])
data

array([[ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.]])

In [230]:
q = data[0:8] + b_12
q

array([[ 0.58058522,  0.58058522,  0.58058522,  0.58058522],
       [-1.6092518 ,  0.3907482 , -1.6092518 ,  0.3907482 ],
       [ 1.35356431,  1.35356431,  1.35356431,  1.35356431],
       [-0.93863738,  1.06136262,  1.06136262,  1.06136262],
       [-1.86803148, -1.86803148, -1.86803148, -1.86803148],
       [ 1.67987524,  1.67987524,  1.67987524,  1.67987524],
       [ 0.86571675,  0.86571675, -1.13428325, -1.13428325],
       [-0.13771591, -0.13771591, -0.13771591,  1.86228409]])

In [238]:
test = np.ones([8,4])
test

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [240]:
np.where(test == -1,q,1)

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [246]:
np.where(data[0:8] == -1,1-q,1)

array([[1.        , 1.        , 1.        , 1.        ],
       [2.6092518 , 1.        , 2.6092518 , 1.        ],
       [1.        , 1.        , 1.        , 1.        ],
       [1.93863738, 1.        , 1.        , 1.        ],
       [2.86803148, 2.86803148, 2.86803148, 2.86803148],
       [1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 2.13428325, 2.13428325],
       [1.13771591, 1.13771591, 1.13771591, 1.        ]])

In [234]:
np.cumprod(np.where(data[0:8] == 1,q,1),axis=0)

array([[0.58058522, 0.58058522, 0.58058522, 0.58058522],
       [0.58058522, 0.22686263, 0.58058522, 0.22686263],
       [0.78585943, 0.30707316, 0.78585943, 0.30707316],
       [0.78585943, 0.32591597, 0.83408183, 0.32591597],
       [0.78585943, 0.32591597, 0.83408183, 0.32591597],
       [1.32014581, 0.54749817, 1.40115341, 0.54749817],
       [1.14287234, 0.47397834, 1.40115341, 0.54749817],
       [1.14287234, 0.47397834, 1.40115341, 1.01959713]])

In [241]:
Q = np.cumprod(np.where(data[0:8] == 1,q,1),axis=0)[-1]
Q

array([1.14287234, 0.47397834, 1.40115341, 1.01959713])

In [242]:
Q * Q

array([1.30615719, 0.22465547, 1.96323088, 1.03957831])

In [244]:
1.01959713 * 1.01959713

1.0395783075042369

In [299]:
# Find a cause

q_2_batch,q_3_batch,q_4_batch,x_w_batch,y_w_batch,z_w_batch,Q_batch = wake_forward_batch(data_batch,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,n_x,n_y,n_z,batch_size)
print ("q_2_batch: " + str(q_2_batch))
print ("q_3_batch: " + str(q_3_batch))
print ("q_4_batch: " + str(q_4_batch))
print ("x_w_batch: " + str(x_w_batch))
print ("y_w_batch: " + str(y_w_batch))
print ("z_w_batch: " + str(z_w_batch))
print ("Q_batch: " + str(Q_batch))

q_2_batch: [[0.13352257 0.79035505 0.98883703 0.98237116 0.99225912 0.77617067
  0.3621196  0.16477452 0.88611348 0.55156464]
 [0.746527   0.04792253 0.50818526 0.3068445  0.74276523 0.17770593
  0.34952719 0.44575866 0.58502072 0.73118235]
 [0.81201678 0.75421757 0.96285477 0.80880274 0.68357803 0.56206191
  0.40901695 0.13934768 0.83461515 0.3390204 ]
 [0.13464848 0.38146688 0.31631035 0.731364   0.27452364 0.08803513
  0.02125021 0.167525   0.0592238  0.27473983]
 [0.22199253 0.71819009 0.66391284 0.41340507 0.69961473 0.92588038
  0.68739074 0.7624029  0.85650132 0.83296682]
 [0.96318306 0.0910634  0.04831625 0.03502429 0.21391472 0.14504926
  0.45693027 0.8295328  0.29473425 0.40728977]
 [0.51277176 0.97257582 0.80054724 0.96441287 0.7970683  0.56828267
  0.36501039 0.27100331 0.643111   0.10218456]
 [0.96457059 0.78971516 0.88459607 0.67839754 0.56306374 0.08005665
  0.03987896 0.53920089 0.42681947 0.37112504]]
q_3_batch: [[0.25408314 0.00922898 0.05513893 0.09196148 0.01157535 

In [289]:
def sleep_forward(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d):
    p_4 = sigmoid(Theta)
    z = ((p_4 > np.random.rand(n_z,1)).astype(int) - 0.5)*2    # rejection sampling
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    y = ((p_3 > np.random.rand(n_y,1)).astype(int) - 0.5)*2
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    x = ((p_2 > np.random.rand(n_x,1)).astype(int) - 0.5)*2
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    d = ((p_1 > np.random.rand(n_d,1)).astype(int) - 0.5)*2
    
    P_4 = (np.cumprod(p_4[np.where(z == 1)])[-1] if p_4[np.where(z == 1)].size != 0 else 1)* (np.cumprod(1-p_4[np.where(z == -1)])[-1] if p_4[np.where(z == -1)].size != 0 else 1)*(1.5**n_z)
    P_3 = (np.cumprod(p_3[np.where(y == 1)])[-1] if p_3[np.where(y == 1)].size != 0 else 1)* (np.cumprod(1-p_3[np.where(y == -1)])[-1] if p_3[np.where(y == -1)].size != 0 else 1)*(1.5**n_y)
    P_2 = (np.cumprod(p_2[np.where(x == 1)])[-1] if p_2[np.where(x == 1)].size != 0 else 1)* (np.cumprod(1-p_2[np.where(x == -1)])[-1] if p_2[np.where(x == -1)].size != 0 else 1)*(1.5**n_x)
    P_1 = (np.cumprod(p_1[np.where(d == 1)])[-1] if p_1[np.where(d == 1)].size != 0 else 1)* (np.cumprod(1-p_1[np.where(d == -1)])[-1] if p_1[np.where(d == -1)].size != 0 else 1)*(1.5**n_d)
    
    P = P_1 * P_2 * P_3 * P_4
    
    return p_4,p_3,p_2,p_1,z,y,x,d,P

In [290]:
# Generate an instance

p_4,p_3,p_2,p_1,z_s,y_s,x_s,d_s,P = sleep_forward(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d)
print ("p_4: " + str(p_4))
print ("p_3: " + str(p_3))
print ("p_2: " + str(p_2))
print ("p_1: " + str(p_1))
print ("z_s: " + str(z_s))
print ("y_s: " + str(y_s))
print ("x_s: " + str(x_s))
print ("d_s: " + str(d_s))
print ("P: " + str(P))

p_4: [[0.63203445]
 [0.49326315]
 [0.56659748]]
p_3: [[0.62801346]
 [0.84892491]
 [0.59614931]
 [0.85701683]
 [0.09015861]]
p_2: [[0.47188771]
 [0.07956516]
 [0.03854843]
 [0.82962015]
 [0.679145  ]
 [0.40547253]
 [0.58197962]
 [0.13779183]]
p_1: [[0.04214507]
 [0.3766529 ]
 [0.25649506]
 [0.70342562]
 [0.87176629]
 [0.66362542]
 [0.02455951]
 [0.54279007]
 [0.30544408]
 [0.43694914]]
z_s: [[1.]
 [1.]
 [1.]]
y_s: [[-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]]
x_s: [[-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]]
d_s: [[-1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]]
P: 0.004736622884412791


In [291]:
def sleep_forward_batch(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d,batch_size):
    p_4 = sigmoid(Theta)
    z = ((p_4 > np.random.rand(n_z,batch_size)).astype(int) - 0.5)*2    # rejection sampling
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    y = ((p_3 > np.random.rand(n_y,batch_size)).astype(int) - 0.5)*2
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    x = ((p_2 > np.random.rand(n_x,batch_size)).astype(int) - 0.5)*2
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    d = ((p_1 > np.random.rand(n_d,batch_size)).astype(int) - 0.5)*2
    
    P_4 = np.cumprod(np.where(z == 1,p_4,1),axis=0)[-1] * np.cumprod(np.where(z == -1,1-p_4,1),axis=0)[-1]*(1.5**n_z)
    P_3 = np.cumprod(np.where(y == 1,p_3,1),axis=0)[-1] * np.cumprod(np.where(y == -1,1-p_3,1),axis=0)[-1]*(1.5**n_y)
    P_2 = np.cumprod(np.where(x == 1,p_2,1),axis=0)[-1] * np.cumprod(np.where(x == -1,1-p_2,1),axis=0)[-1]*(1.5**n_x)
    P_1 = np.cumprod(np.where(d == 1,p_1,1),axis=0)[-1] * np.cumprod(np.where(d == -1,1-p_1,1),axis=0)[-1]*(1.5**n_d)
    
    P = P_1 * P_2 * P_3 * P_4
    
    return p_4,p_3,p_2,p_1,z,y,x,d,P

In [292]:
# Generate an instance

p_4_batch,p_3_batch,p_2_batch,p_1_batch,z_s_batch,y_s_batch,x_s_batch,d_s_batch,P_batch = sleep_forward_batch(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d,batch_size)
print ("p_4_batch: " + str(p_4_batch))
print ("p_3_batch: " + str(p_3_batch))
print ("p_2_batch: " + str(p_2_batch))
print ("p_1_batch: " + str(p_1_batch))
print ("z_s_batch: " + str(z_s_batch))
print ("y_s_batch: " + str(y_s_batch))
print ("x_s_batch: " + str(x_s_batch))
print ("d_s_batch: " + str(d_s_batch))
print ("P_batch: " + str(P_batch))

p_4_batch: [[0.63203445]
 [0.49326315]
 [0.56659748]]
p_3_batch: [[0.31449084 0.62801346 0.74877583 0.47360657 0.47360657 0.20626352
  0.31449084 0.46261216 0.32778483 0.32778483]
 [0.16433769 0.84892491 0.50954429 0.86511485 0.86511485 0.5154228
  0.16433769 0.14697254 0.4823738  0.4823738 ]
 [0.32440578 0.59614931 0.18419335 0.30948017 0.30948017 0.75842059
  0.32440578 0.61263434 0.91181791 0.91181791]
 [0.4903697  0.85701683 0.76732293 0.85165595 0.85165595 0.63620919
  0.4903697  0.50113907 0.64612076 0.64612076]
 [0.68620043 0.09015861 0.3633967  0.22383027 0.22383027 0.27515307
  0.68620043 0.42903207 0.11538792 0.11538792]]
p_2_batch: [[0.46166587 0.47188771 0.73799727 0.46166587 0.54372344 0.54372344
  0.71867251 0.60193744 0.47188771 0.66851003]
 [0.29208864 0.07956516 0.1972312  0.29208864 0.05027019 0.05027019
  0.67689802 0.5397855  0.07956516 0.41799337]
 [0.53599041 0.03854843 0.36561274 0.53599041 0.07654206 0.07654206
  0.75769749 0.42284534 0.03854843 0.60231975]
 [0.

#### Update Parameters

In [293]:
def wake_update_delta(z,y,x,d,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr):
    p_4 = sigmoid(Theta)
    Theta -= lr * (p_4 - (1+z)/2)
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    Theta_43 -= lr * np.outer((p_3 - (1+y)/2), z)
    b_43 -= lr * (p_3 - (1+y)/2)
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    Theta_32 -= lr * np.outer((p_2 - (1+x)/2), y)
    b_32 -= lr * (p_2 - (1+x)/2)
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    Theta_21 -= lr * np.outer((p_1 - (1+d)/2), x)
    b_21 -= lr * (p_1 - (1+d)/2)
    
    return Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21

In [302]:
Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21 = wake_update_delta(z_w,y_w,x_w,data,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr)
print ("Theta: " + str(Theta))
print ("Theta_43: " + str(Theta_43))
print ("Theta_32: " + str(Theta_32))
print ("Theta_21: " + str(Theta_21))
print ("b_43: " + str(b_43))
print ("b_32: " + str(b_32))
print ("b_21: " + str(b_21))

Theta: [[ 0.50949988]
 [-0.00173853]
 [ 0.23979132]]
Theta_43: [[-0.30315111  0.63991859  0.29573185]
 [ 0.82157972  0.92079014 -0.08854866]
 [ 0.94201097 -0.97648757  0.59919061]
 [ 0.28688743  0.60619974  0.00969136]
 [-0.84489762 -0.16805162 -0.50348183]]
Theta_32: [[-0.03350262  0.10787629 -0.39404332  0.13098721  0.72579538]
 [ 0.51976012 -0.21590251 -0.79688277 -0.50699173  0.53781418]
 [ 0.84619338  0.37517995 -0.98165793 -0.47105457  0.84900712]
 [ 0.18493203 -0.84030241  0.84048364 -0.45155773 -0.10385058]
 [-0.62271958  0.40621535  0.47885006  0.06763011  0.72085331]
 [ 0.42505334 -0.81615034 -0.50922385 -0.54322289 -0.05104634]
 [-0.76019336  0.3614176   0.8109769   0.2789625   0.1753973 ]
 [ 0.24262985  0.97166337 -0.34256161  0.2322229   0.23259692]]
Theta_21: [[ 0.83094733  0.69207359  0.85277555 -0.10722073 -0.66738568 -0.93069821
   0.65476792 -0.42750917]
 [ 0.89587527 -0.52345584 -0.17705334 -0.04116675 -0.58665611  0.19993786
   0.73203105 -0.59777834]
 [ 0.83923087 

In [346]:
def wake_update_delta_batch(z,y,x,d,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr,batch_size):
    p_4 = sigmoid(Theta)
    Theta -= lr * np.mean(p_4 - (1+z)/2, axis = 1,keepdims = True)
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    Theta_43 -= lr * np.matmul((p_3 - (1+y)/2), np.transpose(z))/batch_size
    b_43 -= lr * np.mean(p_3 - (1+y)/2, axis = 1,keepdims = True)
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    Theta_32 -= lr * np.matmul((p_2 - (1+x)/2), np.transpose(y))/batch_size
    b_32 -= lr * np.mean(p_2 - (1+x)/2, axis = 1,keepdims = True)
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    Theta_21 -= lr * np.matmul((p_1 - (1+d)/2), np.transpose(x))/batch_size
    b_21 -= lr * np.mean(p_1 - (1+d)/2, axis = 1,keepdims = True)
    
    return Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21

In [267]:
(1+z_s)/2

array([[1., 1., 1., 1., 1., 1., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 1., 1., 1.]])

In [268]:
p_4

array([[0.67624414],
       [0.28825566],
       [0.69786985]])

In [270]:
p_4 - (1+z_s)/2

array([[-0.32375586, -0.32375586, -0.32375586, -0.32375586, -0.32375586,
        -0.32375586,  0.67624414, -0.32375586,  0.67624414, -0.32375586],
       [ 0.28825566,  0.28825566,  0.28825566,  0.28825566,  0.28825566,
         0.28825566,  0.28825566,  0.28825566,  0.28825566,  0.28825566],
       [ 0.69786985, -0.30213015, -0.30213015, -0.30213015, -0.30213015,
        -0.30213015,  0.69786985, -0.30213015, -0.30213015, -0.30213015]])

In [306]:
np.mean(p_4 - (1+z_s)/2, axis = 1,keepdims = True)

array([[-0.36796555],
       [-0.50673685],
       [-0.43340252]])

In [304]:
lr = 0.01

In [347]:
Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21 = wake_update_delta_batch(z_w_batch,y_w_batch,x_w_batch,data_batch,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr,batch_size)
print ("Theta: " + str(Theta))
print ("Theta_43: " + str(Theta_43))
print ("Theta_32: " + str(Theta_32))
print ("Theta_21: " + str(Theta_21))
print ("b_43: " + str(b_43))
print ("b_32: " + str(b_32))
print ("b_21: " + str(b_21))

Theta: [[0.43475421]
 [0.05498185]
 [0.1793641 ]]
Theta_43: [[-0.31863751  0.61809282  0.24078584]
 [ 0.76453844  0.94369926 -0.16423912]
 [ 0.92459553 -0.94527775  0.61953759]
 [ 0.26897673  0.60527937 -0.03230998]
 [-0.77481102 -0.21862447 -0.41869353]]
Theta_32: [[-0.12690143  0.0174195  -0.27592045  0.02991207  0.80870596]
 [ 0.58125748 -0.15270927 -0.84070279 -0.40643257  0.47100116]
 [ 0.83681425  0.40173974 -0.99314566 -0.47716627  0.81183735]
 [ 0.15018029 -0.87763364  0.8530995  -0.44910561 -0.04917912]
 [-0.63651125  0.37415679  0.53696997  0.0312662   0.73620049]
 [ 0.4722166  -0.7343837  -0.56665039 -0.46809698 -0.10975856]
 [-0.77523534  0.36036057  0.81018843  0.22612135  0.15335709]
 [ 0.22657244  0.99860072 -0.35543653  0.22780414  0.22860252]]
Theta_21: [[ 0.78568003  0.72584663  0.90472046 -0.17068041 -0.699972   -0.86960011
   0.63150702 -0.38780264]
 [ 0.76210049 -0.39620425 -0.07159881 -0.17697401 -0.69005036  0.31247972
   0.61702125 -0.49780604]
 [ 0.89719786 -0.

In [39]:
sigmoid(Theta)- (1+z_w)/2

array([-0.27753662,  0.696079  ,  0.71544976])

In [309]:
def sleep_update_delta(z,y,x,d,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr):
    
    q_2 = sigmoid(np.matmul(Phi_12,d) + b_12)
    Phi_12 -= lr * np.outer((q_2 - (1+x)/2), d)
    b_12 -= lr * (q_2 - (1+x)/2)
    
    q_3 = sigmoid(np.matmul(Phi_23,x) + b_23)
    Phi_23 -= lr * np.outer((q_3 - (1+y)/2), x)
    b_23 -= lr * (q_3 - (1+y)/2)
    
    q_4 = sigmoid(np.matmul(Phi_34,y) + b_34)
    Phi_34 -= lr * np.outer((q_4 - (1+z)/2), y)
    b_34 -= lr * (q_4 - (1+z)/2)
    
    return Phi_12,Phi_23,Phi_34,b_12,b_23,b_34

In [311]:
Phi_12,Phi_23,Phi_34,b_12,b_23,b_34 = sleep_update_delta(z_s,y_s,x_s,d_s,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr)
print ("Phi_12: " + str(Phi_12))
print ("Phi_23: " + str(Phi_23))
print ("Phi_34: " + str(Phi_34))
print ("b_12: " + str(b_12))
print ("b_23: " + str(b_23))
print ("b_34: " + str(b_34))

Phi_12: [[ 0.7180436   0.64579115  0.79404802  0.89747906 -0.72757485 -0.24650451
  -0.41404394 -0.26443358  0.95170895 -0.58133454]
 [-0.28463185  0.88455642 -0.37461593  0.59210627  0.0662246  -0.58985912
  -0.4607278   0.01885603 -0.13648134 -0.54849302]
 [ 0.30951642  0.4815223   0.48336166  0.56766155 -0.19486279 -0.35947379
   0.42488659 -0.8352476  -0.01706178 -0.79850034]
 [-0.39607215 -0.46332444 -0.85750949  0.31296711 -0.72020369 -0.10808524
  -0.42284269 -0.52997867  0.46732212  0.1879401 ]
 [-0.57502039 -0.34338057  0.43211552  0.22488259 -0.36673549 -0.44005689
   0.85872404  0.94992842  0.39705357 -0.1325551 ]
 [ 0.70976576  0.6533477   0.09705663 -0.96428008  0.88818583 -0.65245246
  -0.48554959  0.38287921 -0.67110291 -0.53828901]
 [ 0.50736453 -0.50367323 -0.22583004 -0.59791656 -0.59295569  0.95565341
  -0.45123538 -0.47424631 -0.07383352  0.02166803]
 [ 0.36202871  0.69581243 -0.99886593 -0.34544869 -0.59791542 -0.11406172
  -0.05067471 -0.94859625  0.2171699  -0.84

In [344]:
def sleep_update_delta_batch(z,y,x,d,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr,batch_size):
    
    q_2 = sigmoid(np.matmul(Phi_12,d) + b_12)
    Phi_12 -= lr * np.matmul((q_2 - (1+x)/2), np.transpose(d))/batch_size
    b_12 -= lr * np.mean(q_2 - (1+x)/2, axis = 1,keepdims = True)
    
    q_3 = sigmoid(np.matmul(Phi_23,x) + b_23)
    Phi_23 -= lr * np.matmul((q_3 - (1+y)/2), np.transpose(x))/batch_size
    b_23 -= lr * np.mean(q_3 - (1+y)/2, axis = 1,keepdims = True)
    
    q_4 = sigmoid(np.matmul(Phi_34,y) + b_34)
    Phi_34 -= lr * np.matmul((q_4 - (1+z)/2), np.transpose(y))/batch_size
    b_34 -= lr * np.mean(q_4 - (1+z)/2, axis = 1,keepdims = True)
    
    return Phi_12,Phi_23,Phi_34,b_12,b_23,b_34

In [345]:
Phi_12,Phi_23,Phi_34,b_12,b_23,b_34 = sleep_update_delta_batch(z_s_batch,y_s_batch,x_s_batch,d_s_batch,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr,batch_size)
print ("Phi_12: " + str(Phi_12))
print ("Phi_23: " + str(Phi_23))
print ("Phi_34: " + str(Phi_34))
print ("b_12: " + str(b_12))
print ("b_23: " + str(b_23))
print ("b_34: " + str(b_34))

Phi_12: [[ 7.05062085e-01  6.26109636e-01  7.95101731e-01  9.01460125e-01
  -7.22715761e-01 -2.41999691e-01 -3.92321226e-01 -2.80144023e-01
   9.22596186e-01 -5.65447121e-01]
 [-2.59492768e-01  8.64642784e-01 -3.47644356e-01  5.61948185e-01
   1.46938958e-02 -6.16103920e-01 -4.50281672e-01  4.04434153e-02
  -1.57637400e-01 -4.99793406e-01]
 [ 3.06819658e-01  4.77456637e-01  4.90890487e-01  5.84753177e-01
  -2.04430540e-01 -3.69935912e-01  4.38153998e-01 -8.05687912e-01
  -5.11618922e-04 -7.98251515e-01]
 [-3.96153718e-01 -4.75319429e-01 -8.65163619e-01  3.33988180e-01
  -6.67090560e-01 -8.65714030e-02 -4.22226444e-01 -5.76939511e-01
   4.74639215e-01  1.51131835e-01]
 [-5.88992178e-01 -3.21328466e-01  3.84291618e-01  2.11385074e-01
  -3.42114043e-01 -4.25293399e-01  8.16706335e-01  9.14071282e-01
   3.68889388e-01 -1.50053257e-01]
 [ 6.71421535e-01  6.56409043e-01  9.70615145e-02 -9.60762427e-01
   8.63109717e-01 -6.51676605e-01 -4.67784182e-01  3.84638260e-01
  -6.84032998e-01 -5.1378

In [315]:
def wake_update_delta_weighted(z,y,x,d,Q,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr):
    
    p_4 = sigmoid(Theta)
    Theta -= lr * Q * (p_4 - (1+z)/2)
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    Theta_43 -= lr * Q * np.outer((p_3 - (1+y)/2), z)
    b_43 -= lr * Q * (p_3 - (1+y)/2)
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    Theta_32 -= lr * Q * np.outer((p_2 - (1+x)/2), y)
    b_32 -= lr * Q * (p_2 - (1+x)/2)
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    Theta_21 -= lr * Q * np.outer((p_1 - (1+d)/2), x)
    b_21 -= lr * Q * (p_1 - (1+d)/2)
    
    return Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21

In [317]:
Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21 = wake_update_delta_weighted(z_w,y_w,x_w,data,Q,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr)
print ("Theta: " + str(Theta))
print ("Theta_43: " + str(Theta_43))
print ("Theta_32: " + str(Theta_32))
print ("Theta_21: " + str(Theta_21))
print ("b_43: " + str(b_43))
print ("b_32: " + str(b_32))
print ("b_21: " + str(b_21))

Theta: [[0.43783163]
 [0.05411711]
 [0.17880994]]
Theta_43: [[-0.32109     0.62390282  0.24186011]
 [ 0.7652943   0.94602154 -0.16171953]
 [ 0.92679798 -0.94872408  0.61839683]
 [ 0.26822875  0.60771457 -0.03089561]
 [-0.77510713 -0.2200716  -0.42044941]]
Theta_32: [[-0.12552554  0.01868611 -0.27948336  0.03227103  0.8082215 ]
 [ 0.58147541 -0.15278136 -0.84250851 -0.40990849  0.47127415]
 [ 0.84011704  0.40182764 -0.99458729 -0.47420398  0.81268766]
 [ 0.1513269  -0.87625193  0.85395959 -0.45120188 -0.05207736]
 [-0.63887374  0.37343871  0.53536224  0.03102853  0.73842034]
 [ 0.47342805 -0.73632974 -0.56696267 -0.46957496 -0.10992736]
 [-0.77504042  0.35934811  0.81140791  0.22973107  0.15644112]
 [ 0.23035161  0.99855896 -0.35663509  0.23048163  0.22658106]]
Theta_21: [[ 0.78584562  0.726584    0.90397321 -0.16891115 -0.70091469 -0.87116418
   0.62983123 -0.38752565]
 [ 0.76533934 -0.39916303 -0.07236738 -0.17353806 -0.68944906  0.31099285
   0.61883392 -0.4985207 ]
 [ 0.89467764 -0.

In [334]:
Q_batch

array([1.10953444e+00, 3.02487739e-02, 3.36187353e-01, 8.67054206e+00,
       1.52548380e-01, 1.10064204e+00, 3.03376481e-02, 1.61824925e-01,
       2.46639401e-03, 4.23743874e-03])

In [336]:
np.matmul(Q_batch.reshape((-1,2)),b_21.reshape((2,5)))

array([[ 1.13287303e+00,  3.29670528e-01, -1.06906892e-01,
        -2.57993765e-01,  9.81379221e-01],
       [-1.54212402e+00,  1.90175442e+00, -2.87949893e+00,
        -4.62243977e+00,  3.48379909e+00],
       [-8.29220519e-02,  2.73432224e-01, -3.75126651e-01,
        -6.10750823e-01,  5.38314791e-01],
       [-4.06974766e-03,  4.25071738e-02, -5.58451893e-02,
        -9.15231857e-02,  8.60631985e-02],
       [ 1.61051705e-03,  1.60037249e-03, -1.60844233e-03,
        -2.76142821e-03,  3.71569513e-03]])

In [338]:
Q_batch.reshape((1,-1)) * y_w_batch

array([[-1.10953444e+00, -3.02487739e-02, -3.36187353e-01,
        -8.67054206e+00, -1.52548380e-01,  1.10064204e+00,
         3.03376481e-02,  1.61824925e-01,  2.46639401e-03,
         4.23743874e-03],
       [ 1.10953444e+00, -3.02487739e-02,  3.36187353e-01,
        -8.67054206e+00,  1.52548380e-01, -1.10064204e+00,
         3.03376481e-02,  1.61824925e-01,  2.46639401e-03,
        -4.23743874e-03],
       [-1.10953444e+00,  3.02487739e-02,  3.36187353e-01,
        -8.67054206e+00,  1.52548380e-01,  1.10064204e+00,
        -3.03376481e-02, -1.61824925e-01, -2.46639401e-03,
        -4.23743874e-03],
       [-1.10953444e+00, -3.02487739e-02, -3.36187353e-01,
        -8.67054206e+00, -1.52548380e-01, -1.10064204e+00,
        -3.03376481e-02,  1.61824925e-01, -2.46639401e-03,
         4.23743874e-03],
       [-1.10953444e+00, -3.02487739e-02, -3.36187353e-01,
         8.67054206e+00,  1.52548380e-01, -1.10064204e+00,
        -3.03376481e-02, -1.61824925e-01, -2.46639401e-03,
        -4.

In [348]:
def wake_update_delta_weighted_batch(z,y,x,d,Q,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr,batch_size):
    
    p_4 = sigmoid(Theta)
    Theta -= lr *  np.matmul(p_4 - (1+z)/2, Q.reshape((-1,1)))/batch_size
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    Theta_43 -= lr * np.matmul(Q.reshape((1,-1)) * (p_3 - (1+y)/2), np.transpose(z))/batch_size
    b_43 -= lr * np.matmul(p_3 - (1+y)/2, Q.reshape((-1,1)))/batch_size
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    Theta_32 -= lr * np.matmul(Q.reshape((1,-1)) * (p_2 - (1+x)/2), np.transpose(y))/batch_size
    b_32 -= lr * np.matmul(p_2 - (1+x)/2, Q.reshape((-1,1)))/batch_size
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    Theta_21 -= lr * np.matmul(Q.reshape((1,-1)) * (p_1 - (1+d)/2), np.transpose(x))/batch_size
    b_21 -= lr * np.matmul(p_1 - (1+d)/2, Q.reshape((-1,1)))/batch_size
    
   
    return Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21

In [352]:
Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21 = wake_update_delta_weighted_batch(z_w_batch,y_w_batch,x_w_batch,data_batch,Q_batch,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr,batch_size)
print ("Theta: " + str(Theta))
print ("Theta_43: " + str(Theta_43))
print ("Theta_32: " + str(Theta_32))
print ("Theta_21: " + str(Theta_21))
print ("b_43: " + str(b_43))
print ("b_32: " + str(b_32))
print ("b_21: " + str(b_21))

Theta: [[0.42992643]
 [0.05932647]
 [0.18303467]]
Theta_43: [[-0.31168101  0.60978991  0.23555978]
 [ 0.76847589  0.93971518 -0.16967594]
 [ 0.92563398 -0.94685484  0.61817116]
 [ 0.27434207  0.59788042 -0.03869525]
 [-0.7808173  -0.21281274 -0.41242933]]
Theta_32: [[-0.12689302  0.01517119 -0.27525438  0.0286434   0.80967307]
 [ 0.58754118 -0.14605329 -0.83403712 -0.3992268   0.46514131]
 [ 0.83378008  0.40099076 -0.99520208 -0.4794039   0.81346881]
 [ 0.14681643 -0.88017976  0.84916965 -0.45037394 -0.04475884]
 [-0.62862     0.37947019  0.54481441  0.03776513  0.72947764]
 [ 0.47789583 -0.72588119 -0.5610109  -0.46057192 -0.11625308]
 [-0.78218808  0.35351884  0.80330854  0.21731019  0.15839775]
 [ 0.21903597  0.99277409 -0.36215403  0.22061234  0.23488071]]
Theta_21: [[ 0.78568116  0.72509693  0.90509595 -0.17117194 -0.70022893 -0.86950219
   0.63244779 -0.38744953]
 [ 0.75365331 -0.38795919 -0.07759159 -0.18395233 -0.68368747  0.32091461
   0.60925885 -0.5037759 ]
 [ 0.89888128 -0.

In [351]:
sigmoid(Theta)- (1+z_w_batch)/2

array([[-0.39299164,  0.60700836,  0.60700836,  0.60700836,  0.60700836,
        -0.39299164,  0.60700836,  0.60700836, -0.39299164,  0.60700836],
       [-0.486258  , -0.486258  , -0.486258  , -0.486258  , -0.486258  ,
         0.513742  ,  0.513742  ,  0.513742  ,  0.513742  , -0.486258  ],
       [ 0.54472119, -0.45527881,  0.54472119, -0.45527881, -0.45527881,
        -0.45527881, -0.45527881,  0.54472119,  0.54472119, -0.45527881]])

In [353]:
def sleep_update_delta_weighted(z,y,x,d,P,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr):
    
    q_2 = sigmoid(np.matmul(Phi_12,d) + b_12)
    Phi_12 -= lr * P * np.outer((q_2 - (1+x)/2), d)
    b_12 -= lr * P * (q_2 - (1+x)/2)
    
    q_3 = sigmoid(np.matmul(Phi_23,x) + b_23)
    Phi_23 -= lr * P * np.outer((q_3 - (1+y)/2), x)
    b_23 -= lr * P * (q_3 - (1+y)/2)
    
    q_4 = sigmoid(np.matmul(Phi_34,y) + b_34)
    Phi_34 -= lr * P * np.outer((q_4 - (1+z)/2), y)
    b_34 -= lr * P * (q_4 - (1+z)/2)
    
    return Phi_12,Phi_23,Phi_34,b_12,b_23,b_34

In [354]:
Phi_12,Phi_23,Phi_34,b_12,b_23,b_34 = sleep_update_delta_weighted(z_s,y_s,x_s,d_s,P,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr)
print ("Phi_12: " + str(Phi_12))
print ("Phi_23: " + str(Phi_23))
print ("Phi_34: " + str(Phi_34))
print ("b_12: " + str(b_12))
print ("b_23: " + str(b_23))
print ("b_34: " + str(b_34))

Phi_12: [[ 7.05102529e-01  6.26069193e-01  7.95061288e-01  9.01419682e-01
  -7.22756204e-01 -2.42040134e-01 -3.92280783e-01 -2.80184466e-01
   9.22555742e-01 -5.65406678e-01]
 [-2.59454401e-01  8.64604417e-01 -3.47682723e-01  5.61909817e-01
   1.46555286e-02 -6.16142287e-01 -4.50243305e-01  4.04050480e-02
  -1.57675767e-01 -4.99755039e-01]
 [ 3.06840199e-01  4.77436096e-01  4.90869947e-01  5.84732636e-01
  -2.04451081e-01 -3.69956453e-01  4.38174539e-01 -8.05708452e-01
  -5.32159765e-04 -7.98230974e-01]
 [-3.96145496e-01 -4.75327652e-01 -8.65171841e-01  3.33979957e-01
  -6.67098783e-01 -8.65796254e-02 -4.22218221e-01 -5.76947734e-01
   4.74630992e-01  1.51140057e-01]
 [-5.89002237e-01 -3.21318406e-01  3.84301678e-01  2.11395134e-01
  -3.42103984e-01 -4.25283340e-01  8.16696276e-01  9.14081342e-01
   3.68899447e-01 -1.50063317e-01]
 [ 6.71388080e-01  6.56442498e-01  9.70949694e-02 -9.60728972e-01
   8.63143172e-01 -6.51643151e-01 -4.67817637e-01  3.84671715e-01
  -6.83999543e-01 -5.1382

In [356]:
def sleep_update_delta_weighted_batch(z,y,x,d,P,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr,batch_size):
    
    q_2 = sigmoid(np.matmul(Phi_12,d) + b_12)
    Phi_12 -= lr * np.matmul(P.reshape((1,-1)) * (q_2 - (1+x)/2), np.transpose(d))/batch_size
    b_12 -= lr * np.matmul(q_2 - (1+x)/2, P.reshape((-1,1)))/batch_size
    
    q_3 = sigmoid(np.matmul(Phi_23,x) + b_23)
    Phi_23 -= lr * np.matmul(P.reshape((1,-1)) * (q_3 - (1+y)/2), np.transpose(x))/batch_size
    b_23 -= lr * np.matmul(q_3 - (1+y)/2, P.reshape((-1,1)))/batch_size
    
    q_4 = sigmoid(np.matmul(Phi_34,y) + b_34)
    Phi_34 -= lr * np.matmul(P.reshape((1,-1)) * (q_4 - (1+z)/2), np.transpose(y))/batch_size
    b_34 -= lr * np.matmul(q_4 - (1+z)/2, P.reshape((-1,1)))/batch_size
    
   
    return Phi_12,Phi_23,Phi_34,b_12,b_23,b_34

In [357]:
Phi_12,Phi_23,Phi_34,b_12,b_23,b_34 = sleep_update_delta_weighted_batch(z_s_batch,y_s_batch,x_s_batch,d_s_batch,P_batch,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr,batch_size)
print ("Phi_12: " + str(Phi_12))
print ("Phi_23: " + str(Phi_23))
print ("Phi_34: " + str(Phi_34))
print ("b_12: " + str(b_12))
print ("b_23: " + str(b_23))
print ("b_34: " + str(b_34))

Phi_12: [[ 7.05332011e-01  6.25898368e-01  7.95235797e-01  9.01178553e-01
  -7.22939537e-01 -2.42190844e-01 -3.92105975e-01 -2.80000338e-01
   9.22773956e-01 -5.65205370e-01]
 [-2.58720339e-01  8.63761124e-01 -3.46857943e-01  5.61198460e-01
   1.38084497e-02 -6.16989980e-01 -4.49418757e-01  4.12462288e-02
  -1.57003217e-01 -4.98910362e-01]
 [ 3.07300612e-01  4.76899924e-01  4.91423047e-01  5.84296137e-01
  -2.05006574e-01 -3.70498822e-01  4.38727723e-01 -8.05151267e-01
  -7.28680693e-05 -7.97683512e-01]
 [-3.96428846e-01 -4.74959305e-01 -8.65561179e-01  3.34274492e-01
  -6.66664187e-01 -8.61870787e-02 -4.22607450e-01 -5.77384922e-01
   4.74347085e-01  1.50719295e-01]
 [-5.89642154e-01 -3.20509379e-01  3.83454513e-01  2.12030191e-01
  -3.41264479e-01 -4.24476907e-01  8.15849190e-01  9.13230435e-01
   3.68212458e-01 -1.50896540e-01]
 [ 6.71592203e-01  6.56089354e-01  9.74879482e-02 -9.60980088e-01
   8.62731089e-01 -6.51997829e-01 -4.67424401e-01  3.85073580e-01
  -6.83726952e-01 -5.1341

### 3. Experiments

Experiment 1: We use 256 generated samples in well-formed set to train a baseline using traditional training mathod proposed in the original paper.

In [359]:
well_formed_set.shape

(256, 10)

In [360]:
train_set = np.transpose(well_formed_set)
train_set.shape

(10, 256)

In [181]:
n_z = 3
n_y = 5
n_x = 8
n_d = 10

In [426]:
# Random initialization

Phi_12 = np.random.rand(n_x,n_d) *2-1
Phi_23 = np.random.rand(n_y,n_x) *2-1
Phi_34 = np.random.rand(n_z,n_y) *2-1
b_12 = np.random.rand(n_x,1) *2-1
b_23 = np.random.rand(n_y,1) *2-1
b_34 = np.random.rand(n_z,1) *2-1

Theta = np.random.rand(n_z,1) *2-1
Theta_43 = np.random.rand(n_y,n_z) *2-1
Theta_32 = np.random.rand(n_x,n_y) *2-1
Theta_21 = np.random.rand(n_d,n_x) *2-1
b_43 = np.random.rand(n_y,1) *2-1
b_32 = np.random.rand(n_x,1) *2-1
b_21 = np.random.rand(n_d,1) *2-1

In [373]:
# Zero initialization

Phi_12 = np.zeros((n_x,n_d))
Phi_23 = np.zeros((n_y,n_x))
Phi_34 = np.zeros((n_z,n_y))
b_12 = np.zeros((n_x,1))
b_23 = np.zeros((n_y,1))
b_34 = np.zeros((n_z,1))

Theta = np.zeros((n_z,1))
Theta_43 = np.zeros((n_y,n_z))
Theta_32 = np.zeros((n_x,n_y))
Theta_21 = np.zeros((n_d,n_x))
b_43 = np.zeros((n_y,1))
b_32 = np.zeros((n_x,1))
b_21 = np.zeros((n_d,1))

In [383]:
print ("Phi_12: " + str(Phi_12))
print ("Phi_23: " + str(Phi_23))
print ("Phi_34: " + str(Phi_34))
print ("b_12: " + str(b_12))
print ("b_23: " + str(b_23))
print ("b_34: " + str(b_34))

Phi_12: [[-0.36269865  0.89407533 -0.78074121 -0.9085969   0.47307659 -0.34640879
   0.08171577  0.66733834  0.18454457 -0.60455422]
 [-0.60592004 -0.91193836  0.63429869  0.40945908 -0.38371037 -0.15356147
  -0.98635784  0.38954447 -0.77398635  0.99572635]
 [ 0.33387673  0.82140252 -0.01810539 -0.53822077 -0.31279443  0.37863088
  -0.55638973 -0.07558082  0.67041084  0.65177607]
 [ 0.27932081  0.34928344 -0.81500528 -0.95974793  0.91912164 -0.18830971
  -0.29256261 -0.08350745 -0.21755799  0.21878296]
 [ 0.54457568  0.51153638  0.29364467 -0.00137909 -0.34237242 -0.17317923
   0.41944222 -0.1848451   0.49615004 -0.30065137]
 [-0.91697482  0.14440993  0.11931615 -0.94889851 -0.66504829 -0.64973526
  -0.30273211 -0.39856389 -0.51251448 -0.72069941]
 [ 0.54245289 -0.56743709 -0.61365155 -0.54640114 -0.56762551 -0.20997043
   0.03250334  0.25735658 -0.38261971 -0.87738157]
 [ 0.57142081 -0.56336828 -0.48998499 -0.36152764 -0.87118957 -0.74620482
  -0.60382577 -0.41740661  0.75410792 -0.80

In [384]:
print ("Theta: " + str(Theta))
print ("Theta_43: " + str(Theta_43))
print ("Theta_32: " + str(Theta_32))
print ("Theta_21: " + str(Theta_21))
print ("b_43: " + str(b_43))
print ("b_32: " + str(b_32))
print ("b_21: " + str(b_21))

Theta: [[-0.97082694]
 [-0.56578044]
 [-0.80671329]]
Theta_43: [[-0.46530804  0.06910619  0.80919499]
 [ 0.51894606 -0.59611203 -0.73200645]
 [-0.21750814  0.38857122  0.75174856]
 [ 0.55688127 -0.05844833 -0.82928573]
 [ 0.17837977  0.11063671  0.81923425]]
Theta_32: [[-0.78137304  0.64239397  0.07243662 -0.25736197  0.11619475]
 [ 0.94162062 -0.07192714 -0.06958583  0.31732883  0.63248108]
 [ 0.56455673  0.01757698 -0.47370764 -0.99182941 -0.83338552]
 [-0.40465956 -0.03710654 -0.49672684 -0.33317992 -0.53654296]
 [-0.62970959 -0.69844212  0.52770212  0.57504246 -0.65145038]
 [ 0.97770136 -0.39003214  0.76222649 -0.87462203 -0.74379525]
 [ 0.23016752  0.47886679 -0.31939635 -0.72761344  0.87212058]
 [-0.34700673 -0.86133984  0.43733292  0.38130027 -0.98057296]]
Theta_21: [[-0.72892024 -0.49753869  0.81181981  0.41622254  0.03771239 -0.650321
  -0.71297175 -0.25102488]
 [-0.51854293 -0.76264014  0.40055592 -0.51373024  0.79439456  0.91378351
  -0.87975956 -0.47856546]
 [-0.34723186 -0

In [427]:
Phi_12_cache = np.copy(Phi_12)
Phi_23_cache = np.copy(Phi_23)
Phi_34_cache = np.copy(Phi_34)
b_12_cache = np.copy(b_12)
b_23_cache = np.copy(b_23)
b_34_cache = np.copy(b_34)

Theta_cache = np.copy(Theta)
Theta_43_cache = np.copy(Theta_43)
Theta_32_cache = np.copy(Theta_32)
Theta_21_cache = np.copy(Theta_21)
b_43_cache = np.copy(b_43)
b_32_cache = np.copy(b_32)
b_21_cache = np.copy(b_21)

In [428]:
lr = 0.01
epoch = 10

In [430]:
# Sample-based training
# Local delta rule
for e in range(epoch):
    for i in range(train_set.shape[1]):
        data = train_set[:,i:i+1]
        q_2,q_3,q_4,x_w,y_w,z_w,Q = wake_forward(data,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,n_x,n_y,n_z)
        Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21 = wake_update_delta(z_w,y_w,x_w,data,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr)
        p_4,p_3,p_2,p_1,z_s,y_s,x_s,d_s,P = sleep_forward(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d)
        Phi_12,Phi_23,Phi_34,b_12,b_23,b_34 = sleep_update_delta(z_s,y_s,x_s,d_s,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr)

    recog_phi = np.sum((Phi_12 - Phi_12_cache)**2) + np.sum((Phi_23 - Phi_23_cache)**2) + np.sum((Phi_34 - Phi_34_cache)**2) 
    recog_b = np.sum((b_12 - b_12_cache)**2) + np.sum((b_23 - b_23_cache)**2) + np.sum((b_34 - b_34_cache)**2)
    gener_theta = np.sum((Theta - Theta_cache)**2) + np.sum((Theta_43 - Theta_43_cache)**2) + np.sum((Theta_32 - Theta_32_cache)**2) + np.sum((Theta_21 - Theta_21_cache)**2)
    gener_b = np.sum((b_43 - b_43_cache)**2) + np.sum((b_32 - b_32_cache)**2) + np.sum((b_21 - b_21_cache)**2)
    print ("recog_phi: " + str(recog_phi))
    print ("recog_b: " + str(recog_b))
    print ("gener_theta: " + str(gener_theta))
    print ("gener_b: " + str(gener_b))
    print('')
    
    Phi_12_cache = np.copy(Phi_12)
    Phi_23_cache = np.copy(Phi_23)
    Phi_34_cache = np.copy(Phi_34)
    b_12_cache = np.copy(b_12)
    b_23_cache = np.copy(b_23)
    b_34_cache = np.copy(b_34)

    Theta_cache = np.copy(Theta)
    Theta_43_cache = np.copy(Theta_43)
    Theta_32_cache = np.copy(Theta_32)
    Theta_21_cache = np.copy(Theta_21)
    b_43_cache = np.copy(b_43)
    b_32_cache = np.copy(b_32)
    b_21_cache = np.copy(b_21)
    
    np.random.shuffle(train_set)


recog_phi: 0.6115233587091153
recog_b: 0.05196038648331658
gener_theta: 1.1053306682011572
gener_b: 0.9325376006996552

recog_phi: 0.45634679846066056
recog_b: 0.05559485845129042
gener_theta: 0.898503641420925
gener_b: 0.6789620287678042

recog_phi: 0.5994211023053237
recog_b: 0.04851267070791189
gener_theta: 0.7526419782306422
gener_b: 0.7534203530989925

recog_phi: 0.4125470852644962
recog_b: 0.05516666588797189
gener_theta: 0.7200607136091748
gener_b: 0.8692110746186464

recog_phi: 0.5892958256253538
recog_b: 0.045378010919591025
gener_theta: 0.8302167794487738
gener_b: 0.7493198769036344

recog_phi: 0.631544570776467
recog_b: 0.029912844901708813
gener_theta: 0.5658562209633506
gener_b: 0.30630593099855363

recog_phi: 0.6452502720943883
recog_b: 0.08453958088742428
gener_theta: 0.6226291394631347
gener_b: 0.2561247260224115

recog_phi: 0.49526285915894763
recog_b: 0.05450681403284786
gener_theta: 0.8898096366508254
gener_b: 0.7483248678991568

recog_phi: 0.4687913835342336
recog_b

In [431]:
print ("Phi_12: " + str(Phi_12))
print ("Phi_23: " + str(Phi_23))
print ("Phi_34: " + str(Phi_34))
print ("b_12: " + str(b_12))
print ("b_12_cache: " + str(b_12_cache))
print ("b_23: " + str(b_23))
print ("b_34: " + str(b_34))
print ("Phi_12_cache: " + str(Phi_12_cache))

Phi_12: [[ 0.25817555  0.3164928  -1.02703235  0.29891417  0.07946648 -0.37401944
  -0.15915869 -0.06772254  0.18613879  0.1644277 ]
 [ 0.36952015  0.10375914 -0.20158691  0.08880081 -0.36772491  0.13143094
  -0.57350717 -0.16661315 -0.41672413  0.75439095]
 [ 0.15901325 -0.21374134  0.32548336  0.65453622  0.33200995 -0.25354196
  -0.41417159 -0.93715205  0.281141    0.17139529]
 [ 0.02581285 -0.40217983 -0.30655154 -0.29362177  0.15646529  0.48912374
  -0.67693102  0.54534813  0.27854937 -0.28270075]
 [ 0.15891551  0.06377039  0.39247674  0.92839659 -0.03732773 -0.27543192
  -0.09174925  0.21518005 -0.49557417 -0.25282927]
 [-0.37051913 -0.0585297  -0.15334527 -0.22364293  0.72180331  0.00891843
  -0.00593579  0.07698222 -0.26579795 -0.0372252 ]
 [-0.23354202 -0.27331169 -0.27278212  0.50034928 -0.11686393  0.25778282
   0.62880932 -0.37494928 -0.22792936 -0.57415906]
 [-0.29712824  0.07653468 -0.20118     0.38876564 -0.19659231  0.08879732
   0.00725532  0.02123035 -0.3228604   0.15

In [381]:
print ("Theta: " + str(Theta))
print ("Theta_43: " + str(Theta_43))
print ("Theta_32: " + str(Theta_32))
print ("Theta_21: " + str(Theta_21))
print ("b_43: " + str(b_43))
print ("b_32: " + str(b_32))
print ("b_21: " + str(b_21))

Theta: [[ 0.10810326]
 [ 0.35930207]
 [-0.31405824]]
Theta_43: [[-0.09575273  0.14007702 -0.39469428]
 [ 0.44650337  0.28564228  0.52337354]
 [ 0.24678362  0.51557779 -0.90670904]
 [ 0.49337953 -0.37200851  0.56883611]
 [-0.24565591  0.22592884  0.76289209]]
Theta_32: [[-0.38991613  0.40975452 -0.25297272 -0.13736693  0.03465881]
 [ 0.16633522  0.42036017 -0.46594193  0.18229528 -0.04399158]
 [ 0.70085579  0.9517073   0.11474516  0.35057096 -0.0607529 ]
 [ 0.34409427 -0.2085122   0.45461835  0.01078571  0.48475153]
 [-0.10280801 -0.3667067   0.03267916  0.24170922 -0.45970471]
 [ 0.04159141  0.28744414 -0.72313816 -0.42226192 -0.16174755]
 [-0.12434905 -0.32674417 -0.21529194 -0.05896704  0.39738625]
 [-0.16669404  0.0938111   0.52784378 -0.08935549 -0.37406569]]
Theta_21: [[ 0.19241651 -0.41044561  0.05779248  0.0997436   0.01223997  0.12874296
  -0.07624183 -0.26322726]
 [-0.786116   -0.07383121  0.69068188  0.03278605  0.49494039 -0.2110568
   0.4087714  -0.33534604]
 [ 0.18399694 -

#### Collection of Functions
1. q_2,q_3,q_4,x_w,y_w,z_w,Q = wake_forward(data,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,n_x,n_y,n_z)
2. p_4,p_3,p_2,p_1,z_s,y_s,x_s,d_s,P = sleep_forward(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d)
3. Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21 = wake_update_delta(z_w,y_w,x_w,data,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr)
4. Phi_12,Phi_23,Phi_34,b_12,b_23,b_34 = sleep_update_delta(z_s,y_s,x_s,d_s,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr)
5. Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21 = wake_update_delta_weighted(z_w,y_w,x_w,data,Q,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr)
6. Phi_12,Phi_23,Phi_34,b_12,b_23,b_34 = sleep_update_delta_weighted(z_s,y_s,x_s,d_s,P,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr)


1. q_2_batch,q_3_batch,q_4_batch,x_w_batch,y_w_batch,z_w_batch,Q_batch = wake_forward_batch(data_batch,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,n_x,n_y,n_z,batch_size)
2. p_4_batch,p_3_batch,p_2_batch,p_1_batch,z_s_batch,y_s_batch,x_s_batch,d_s_batch,P_batch = sleep_forward_batch(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d,batch_size)
3. Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21 = wake_update_delta_batch(z_w_batch,y_w_batch,x_w_batch,data_batch,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr,batch_size)
4. Phi_12,Phi_23,Phi_34,b_12,b_23,b_34 = sleep_update_delta_batch(z_s_batch,y_s_batch,x_s_batch,d_s_batch,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr,batch_size)
5. Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21 = wake_update_delta_weighted_batch(z_w_batch,y_w_batch,x_w_batch,data_batch,Q_batch,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr,batch_size)
6. Phi_12,Phi_23,Phi_34,b_12,b_23,b_34 = sleep_update_delta_weighted_batch(z_s_batch,y_s_batch,x_s_batch,d_s_batch,P_batch,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr,batch_size)

#### After training...

In [496]:
test_size = 100

In [497]:
p_4_batch,p_3_batch,p_2_batch,p_1_batch,z_s_batch,y_s_batch,x_s_batch,d_s_batch,P_batch = sleep_forward_batch(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d,test_size)
print ("p_4_batch: " + str(p_4_batch))
print ("p_3_batch: " + str(p_3_batch))
print ("p_2_batch: " + str(p_2_batch))
print ("p_1_batch: " + str(p_1_batch))
print ("z_s_batch: " + str(z_s_batch))
print ("y_s_batch: " + str(y_s_batch))
print ("x_s_batch: " + str(x_s_batch))
print ("d_s_batch: " + str(d_s_batch))
print ("P_batch: " + str(P_batch))

p_4_batch: [[0.51993293]
 [0.5533909 ]
 [0.51478194]]
p_3_batch: [[0.18033514 0.28154926 0.38576719 0.18033514 0.42953435 0.42953435
  0.15505819 0.42953435 0.18033514 0.57286198 0.42953435 0.3196456
  0.15505819 0.42953435 0.57286198 0.38576719 0.42953435 0.3196456
  0.15505819 0.3196456  0.57286198 0.38576719 0.28154926 0.61654909
  0.28154926 0.15505819 0.18033514 0.42953435 0.28154926 0.18033514
  0.42953435 0.18033514 0.38576719 0.42953435 0.18033514 0.15505819
  0.57286198 0.61654909 0.15505819 0.3196456  0.57286198 0.42953435
  0.28154926 0.28154926 0.15505819 0.18033514 0.15505819 0.3196456
  0.57286198 0.61654909 0.15505819 0.18033514 0.38576719 0.61654909
  0.38576719 0.28154926 0.57286198 0.3196456  0.61654909 0.61654909
  0.57286198 0.15505819 0.57286198 0.18033514 0.3196456  0.3196456
  0.61654909 0.3196456  0.61654909 0.57286198 0.28154926 0.15505819
  0.57286198 0.18033514 0.57286198 0.3196456  0.3196456  0.61654909
  0.28154926 0.15505819 0.3196456  0.28154926 0.1803351

In [498]:
def sleep_forward_batch_deterministic(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d,batch_size):
    p_4 = sigmoid(Theta)
    z = ((p_4 > np.random.rand(n_z,batch_size)).astype(int) - 0.5)*2    # rejection sampling
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    y = ((p_3 > np.random.rand(n_y,batch_size)).astype(int) - 0.5)*2
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    x = ((p_2 > np.random.rand(n_x,batch_size)).astype(int) - 0.5)*2
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    #d = ((p_1 > np.random.rand(n_d,batch_size)).astype(int) - 0.5)*2
    d = np.where(p_1 > 0.5,1,-1)
    
    P_4 = np.cumprod(np.where(z == 1,p_4,1),axis=0)[-1] * np.cumprod(np.where(z == -1,1-p_4,1),axis=0)[-1]*(1.5**n_z)
    P_3 = np.cumprod(np.where(y == 1,p_3,1),axis=0)[-1] * np.cumprod(np.where(y == -1,1-p_3,1),axis=0)[-1]*(1.5**n_y)
    P_2 = np.cumprod(np.where(x == 1,p_2,1),axis=0)[-1] * np.cumprod(np.where(x == -1,1-p_2,1),axis=0)[-1]*(1.5**n_x)
    P_1 = np.cumprod(np.where(d == 1,p_1,1),axis=0)[-1] * np.cumprod(np.where(d == -1,1-p_1,1),axis=0)[-1]*(1.5**n_d)
    
    P = P_1 * P_2 * P_3 * P_4
    
    return p_4,p_3,p_2,p_1,z,y,x,d,P

In [499]:
p_4_batch,p_3_batch,p_2_batch,p_1_batch,z_s_batch,y_s_batch,x_s_batch,d_s_batch,P_batch = sleep_forward_batch_deterministic(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d,test_size)
print ("p_4_batch: " + str(p_4_batch))
print ("p_3_batch: " + str(p_3_batch))
print ("p_2_batch: " + str(p_2_batch))
print ("p_1_batch: " + str(p_1_batch))
print ("z_s_batch: " + str(z_s_batch))
print ("y_s_batch: " + str(y_s_batch))
print ("x_s_batch: " + str(x_s_batch))
print ("d_s_batch: " + str(d_s_batch))
print ("P_batch: " + str(P_batch))

p_4_batch: [[0.51993293]
 [0.5533909 ]
 [0.51478194]]
p_3_batch: [[0.15505819 0.3196456  0.61654909 0.38576719 0.18033514 0.38576719
  0.15505819 0.38576719 0.3196456  0.3196456  0.42953435 0.57286198
  0.38576719 0.3196456  0.42953435 0.3196456  0.28154926 0.18033514
  0.57286198 0.28154926 0.3196456  0.42953435 0.61654909 0.18033514
  0.3196456  0.42953435 0.38576719 0.61654909 0.15505819 0.15505819
  0.3196456  0.15505819 0.42953435 0.15505819 0.38576719 0.42953435
  0.38576719 0.28154926 0.61654909 0.15505819 0.3196456  0.3196456
  0.18033514 0.42953435 0.38576719 0.42953435 0.57286198 0.28154926
  0.28154926 0.61654909 0.18033514 0.42953435 0.3196456  0.42953435
  0.42953435 0.28154926 0.18033514 0.18033514 0.61654909 0.61654909
  0.57286198 0.15505819 0.38576719 0.28154926 0.15505819 0.3196456
  0.15505819 0.57286198 0.28154926 0.28154926 0.28154926 0.28154926
  0.61654909 0.61654909 0.38576719 0.38576719 0.18033514 0.18033514
  0.42953435 0.38576719 0.18033514 0.3196456  0.28154

#### Check with the rules (well-formedness rules)

1. Start with 1
2. Forbid 00100 (no 100, 001 on the boundary)
3. Forbid 0000


In [500]:
test_set = (d_s_batch+1)/2
np.array_equal(test_set[0:3,4], [0,1,1])

False

In [501]:
test_set

array([[0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1.,
        1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
        0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
        1., 1., 1., 0.],
       [1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1.,
        0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0.,
        0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1.,
        1., 1., 1., 1.],
       [0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
        1., 1.

In [502]:
st, counts = np.unique(test_set, axis=1,return_counts=True)
print(st)
print(counts)
st.shape

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.
  1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1.]
 [0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0.
  0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.
  1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
  1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1.
  1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

(10, 51)

In [503]:
violation_rule1 = 0
violation_rule1_index = []
violation_rule2 = 0
violation_rule2_index = []
violation_rule3 = 0
violation_rule3_index = []
valid_num = 0
valid_index = []

for i in range(test_set.shape[1]):
    flag = 0
    pattern = test_set[:,i]
    if pattern[0] != 1:
        violation_rule1 += 1
        violation_rule1_index.append(i)
        flag = 1
    if np.array_equal(pattern[0:3], [1,0,0]) or np.array_equal(pattern[-3:], [0,0,1]):
        violation_rule2 += 1
        violation_rule2_index.append(i)
        flag = 1
    for j in range(5):
        if np.array_equal(pattern[j:j+5],[0,0,1,0,0]):
            violation_rule2 += 1
            violation_rule2_index.append(i)
            flag = 1
            break
    for j in range(6):
        if np.array_equal(pattern[j:j+4],[0,0,0,0]):
            violation_rule3 += 1
            violation_rule3_index.append(i)
            flag = 1
            break
    if flag == 0:
        valid_num += 1
        valid_index.append(i)

In [504]:
print ("violation_rule1: " + str(violation_rule1))
print ("violation_rule1_index: " + str(violation_rule1_index))
print ("violation_rule2: " + str(violation_rule2))
print ("violation_rule2_index: " + str(violation_rule2_index))
print ("violation_rule3: " + str(violation_rule3))
print ("violation_rule3_index: " + str(violation_rule3_index))
print ("valid_num: " + str(valid_num))
print ("valid_index: " + str(valid_index))
print ("valid_percentage: " + str(valid_num/test_size))

violation_rule1: 28
violation_rule1_index: [0, 2, 5, 11, 12, 16, 25, 26, 29, 30, 34, 35, 37, 38, 40, 48, 52, 53, 61, 71, 73, 75, 79, 80, 81, 89, 91, 99]
violation_rule2: 15
violation_rule2_index: [6, 14, 20, 23, 24, 28, 37, 44, 66, 70, 81, 82, 86, 88, 94]
violation_rule3: 1
violation_rule3_index: [53]
valid_num: 59
valid_index: [1, 3, 4, 7, 8, 9, 10, 13, 15, 17, 18, 19, 21, 22, 27, 31, 32, 33, 36, 39, 41, 42, 43, 45, 46, 47, 49, 50, 51, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 65, 67, 68, 69, 72, 74, 76, 77, 78, 83, 84, 85, 87, 90, 92, 93, 95, 96, 97, 98]
valid_percentage: 0.59


In [488]:
test_set[-3:,7]

array([0., 1., 1.])

In [256]:
x = np.arange(8.0)
np.split(x, [3, 5])

[array([0., 1., 2.]), array([3., 4.]), array([5., 6., 7.])]

In [424]:
np.random.shuffle(train_set)

In [425]:
train_set

array([[ 1., -1., -1., ...,  1., -1.,  1.],
       [-1.,  1.,  1., ..., -1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1., -1., -1.],
       ...,
       [-1., -1., -1., ..., -1.,  1.,  1.],
       [-1.,  1.,  1., ..., -1.,  1.,  1.],
       [-1., -1.,  1., ..., -1.,  1.,  1.]])